En esta lección vamos a ver cómo crear un servicio Cortex Search mediante SQL.

Para el siguiente ejemplo vamos a crear la tabla `opiniones_clientes` la cual contiene opiniones de diversos clientes sobre estufas de gas y sobre ella construiremos el servicio de Cortex Search.

In [ ]:
-- Crear el esquema correspondiente a la sección 4
CREATE OR REPLACE SCHEMA curso_ia.seccion_4;


-- DDL para crear la tabla opiniones_clientes
CREATE TABLE curso_ia.seccion_4.opiniones_clientes (
    id_cliente INTEGER,
    estado VARCHAR(50),
    opinion_servicio VARCHAR(500)
);

-- Insertar 7 registros con opiniones variadas sobre estufas de gas
INSERT INTO opiniones_clientes (id_cliente, estado, opinion_servicio) VALUES
(1001, 'Ciudad de México', 'Excelente servicio de instalación de estufa de gas. Los técnicos fueron muy profesionales y explicaron todo el proceso de seguridad.'),
(1002, 'Jalisco', 'La estufa de gas que compramos funciona perfecto, pero el servicio de entrega tardó más de lo prometido. En general satisfecho.'),
(1003, 'Nuevo León', 'Pésima experiencia con el mantenimiento de mi estufa de gas. El técnico llegó tarde y no solucionó completamente el problema de la flama.'),
(1004, 'Puebla', 'Muy contento con mi nueva estufa de gas de 6 quemadores. El personal de ventas fue muy atento y me ayudó a elegir el modelo perfecto.'),
(1005, 'Veracruz', 'El servicio técnico para reparar mi estufa de gas fue rápido y eficiente. Ahora funciona como nueva. Definitivamente recomendaría el servicio.'),
(1006, 'Guanajuato', 'Regular el servicio. La estufa de gas funciona bien pero tuvimos problemas con la facturación y el servicio al cliente no fue muy útil.'),
(1007, 'Sonora', 'Horrible experiencia. La estufa de gas llegó con defectos y cuando solicité el cambio, me hicieron dar muchas vueltas. No volvería a comprar aquí.');

Se requiere el seguimiento de cambios para crear el servicio de búsqueda de Cortex. Deben tener en cuenta que si van a usar un rol que no sea owner de la tabla para crear el servicio de búsqueda, deben ejecutar la siguiente instrucción para habilitar el seguimiento de cambios en la tabla.

In [ ]:
ALTER TABLE curso_ia.seccion_4.opiniones_clientes SET
  CHANGE_TRACKING = TRUE;

### Crear el servicio

Puede crear un servicio de búsqueda de Cortex con una sola consulta SQL. Al crear un servicio de búsqueda de Cortex, Snowflake realiza transformaciones en los datos de origen para prepararlos para un servicio de baja latencia.

El siguiente ejemplo demuestra cómo crear un servicio de búsqueda Cortex con `CREATE CORTEX SEARCH SERVICE` en la tabla que acabamos de crear.


**Nota:** Al crear un servicio de búsqueda, el índice de búsqueda se construye como parte del proceso. Esto significa que la instrucción `CREATE CORTEX SEARCH SERVICE` puede tardar más en completarse para conjuntos de datos más grandes.

Primero crearemos un warehouse dedicado para el servicio de Cortex que crearemos posteriormente.

In [ ]:
-- Crear un warehouse para el servicio de Cortex Search
CREATE OR REPLACE WAREHOUSE search_service_wh
  WAREHOUSE_TYPE = STANDARD
  WAREHOUSE_SIZE =  MEDIUM 
  SCALING_POLICY = STANDARD
  AUTO_SUSPEND = 300
  AUTO_RESUME = TRUE 
  INITIALLY_SUSPENDED = TRUE
  COMMENT = 'Warehouse para el servicio de Cortex Search';

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE opiniones_search_service
  ON opinion_servicio
  ATTRIBUTES estado
  WAREHOUSE = search_service_wh
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'voyage-multilingual-2'
  AS (
    SELECT
        opinion_servicio,
        estado,
        id_cliente
    FROM curso_ia.seccion_4.opiniones_clientes
);

El comando anterior inicia la creación del servicio de búsqueda de nuestros datos. En este ejemplo:

- Las consultas al servicio buscarán coincidencias en la columna `opinion_servicio`.
- El parámetro `TARGET_LAG` indica que el servicio de búsqueda de Cortex comprobará las actualizaciones de la tabla base `opiniones_clientes` aproximadamente una vez al día.
- Las columnas `estado` y `id_cliente` se indexarán para que puedan devolverse junto con los resultados de las consultas en la columna `opiniones_clientes`.
- La columna `estado` estará disponible como columna de filtro al consultar la columna transcript_text.
- El warehouse `compute_wh` se utilizará para materializar los resultados de la consulta especificada inicialmente y cada vez que se modifique la tabla base.

### Notas

- Dependiendo del tamaño del warehouse especificado en la consulta y del número de filas de la tabla, este comando `CREATE` puede tardar varias horas en completarse.
- Snowflake recomienda utilizar un warehouse dedicado de tamaño no superior a `MEDIUM` para cada servicio.
- Las columnas del campo `ATTRIBUTES` deben incluirse en la consulta de origen, ya sea mediante una enumeración explícita o un comodín (*).

### Vista previa del servicio

Para confirmar que el servicio se ha rellenado correctamente con datos, podemos obtener una vista previa del servicio mediante la función `SEARCH_PREVIEW` como se muestra a continuación.

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'curso_ia.seccion_4.opiniones_search_service',
      '{
        "query": "personal de ventas",
        "columns":[
            "opinion_servicio",
            "estado"
        ],
        "filter": {"@eq": {"estado": "Puebla"} },
        "limit":1
      }'
  )
)['results'] as results;

Esta respuesta confirma que el servicio cuenta con datos y ofrece resultados para la consulta proporcionada. También podemos usar la función de tabla `CORTEX_SEARCH_DATA_SCAN` para inspeccionar el contenido del servicio. Veamos un ejemplo:

In [ ]:
SELECT
*
FROM
  TABLE (
    CORTEX_SEARCH_DATA_SCAN (
      SERVICE_NAME => 'opiniones_search_service'
    )
  );